In [ ]:
!pip install opencv-python

In [ ]:
import cv2
import os
import glob 
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import zipfile as zf
import random
from datetime import datetime
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from scipy.fftpack import dct as dct
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
file_path = "Reduced MNIST Data/Reduced Trainging data"
def read_train(file_path):
    #Reading Training Data

    files = os.listdir(file_path)
    #print(files)
    Train_data = []
    for file in files:
        data_path = file_path + '/' + file
        data = os.listdir(data_path)
        for img in data:
            img_array = cv2.imread(os.path.join(data_path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
            #plt.imshow(img_array, cmap='gray')  # graph it
            #plt.show() 
            Train_data.append([img_array, file])
    X = []
    y =[]
    for img,label in Train_data:
        X.append(img)
        y.append(label)
    y = np.array(y)
    return X , y

In [ ]:
X_train,y_train=read_train(file_path)

In [ ]:
test_path = "Reduced MNIST Data/Reduced Testing data"
def read_test(test_path):
    files = os.listdir(test_path)
    Test_data = []
    for file in files:
        data_path = test_path + '/' + file
        data = os.listdir(data_path)
        for img in data:
            img_array = cv2.imread(os.path.join(data_path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
            Test_data.append([img_array, file])
    random.shuffle(Test_data)
    X_test = []
    y_test =[]
    for img,label in Test_data:
        X_test.append(img)
        y_test.append(label)
    
    return X_test , y_test

In [ ]:
X_test,y_test=read_test(test_path)
X_train,y_train=read_train(file_path)


In [ ]:
def zigzag(input):
    #initializing the variables
    #----------------------------------
    h = 0
    v = 0

    vmin = 0
    hmin = 0

    vmax = input.shape[0]
    hmax = input.shape[1]
    
    #print(vmax ,hmax )

    i = 0

    output = np.zeros(( vmax * hmax))
    #----------------------------------

    while ((v < vmax) and (h < hmax)):
    	
        if ((h + v) % 2) == 0:                 # going up
            
            if (v == vmin):
            	#print(1)
                output[i] = input[v, h]        # if we got to the first line

                if (h == hmax):
                    v = v + 1
                else:
                    h = h + 1                        

                i = i + 1

            elif ((h == hmax -1 ) and (v < vmax)):   # if we got to the last column
            	#print(2)
            	output[i] = input[v, h] 
            	v = v + 1
            	i = i + 1

            elif ((v > vmin) and (h < hmax -1 )):    # all other cases
            	#print(3)
            	output[i] = input[v, h] 
            	v = v - 1
            	h = h + 1
            	i = i + 1

        
        else:                                    # going down

        	if ((v == vmax -1) and (h <= hmax -1)):       # if we got to the last line
        		#print(4)
        		output[i] = input[v, h] 
        		h = h + 1
        		i = i + 1
        
        	elif (h == hmin):                  # if we got to the first column
        		#print(5)
        		output[i] = input[v, h] 

        		if (v == vmax -1):
        			h = h + 1
        		else:
        			v = v + 1

        		i = i + 1

        	elif ((v < vmax -1) and (h > hmin)):     # all other cases
        		#print(6)
        		output[i] = input[v, h] 
        		v = v + 1
        		h = h - 1
        		i = i + 1




        if ((v == vmax-1) and (h == hmax-1)):          # bottom right element
        	#print(7)        	
        	output[i] = input[v, h] 
        	break

    #print ('v:',v,', h:',h,', i:',i)
    return output


In [ ]:
y_test = np.array(y_test)
y_test = y_test.reshape(2000,-1)

In [ ]:
def train_SelectKBest(X,X_test,y,k=262):
    X = np.array(X)
    X_train = X.reshape(10000,-1)
    y = np.array(y)
    y_train = y.reshape(10000,-1)
    X_test = np.array(X_test)
    X_test = X_test.reshape(2000,-1)

    selector = SelectKBest(chi2, k=k)
    selector.fit(X_train, y_train)
    X_train = selector.transform(X_train)
    X_test = selector.transform(X_test)
    return X_train,X_test

In [ ]:
def train_DCT(X,X_test):
    X = np.array(X)

    X_train = []
    for i in X:
        X_train.append(zigzag(dct(i)))
    X_train=np.array(X_train)
    x_train =[]
    for j in X_train:
        x_train.append(j[:180])
    
    x_train = np.array(x_train)
    x_test = []
    for i in X_test:
        x_test.append(zigzag(dct(i)))
    X_test = np.array(x_test)
    x_test =[]
    for j in X_test:
        x_test.append(j[:180])
    x_test = np.array(x_test)
    
    
    return x_train,x_test

In [ ]:
def train_PCA(X,X_test):
    X = np.array(X)
    X_train = X.reshape(10000,-1)
    X_test = np.array(X_test)
    X_test = X_test.reshape(2000,-1)
    pca = PCA(n_components = 0.95)
    X_train=pca.fit_transform(X_train)
    X_test  = pca.transform(X_test)
    return X_train,X_test

In [ ]:
X_train_pca,X_test_pca= train_PCA(X_train,X_test)
print(X_train_pca.shape)
X_train_SKB,X_test_SKB= train_SelectKBest(X_train,X_test,y_train,k=220)
print(X_train_SKB.shape)
X_train_DCT,X_test_DCT= train_DCT(X_train,X_test)
print(X_train_DCT.shape)

(10000, 262)
(10000, 220)
(10000, 180)


In [ ]:
matrix =[('K-means-Clustering',  1),
            ('K-means-Clustering',  4),
            ('K-means-Clustering', 16),
            ('GMM',  1),
            ('GMM',  4),
            ('GMM', 16),
            ('SVM','Linear'),
            ('SVM','NonLinear')]
Indices =pd.MultiIndex.from_tuples(matrix)
columns=[('DCT','PCA','SelectKBest'),('Accuracy','ProcessingTime')]
Columns=pd.MultiIndex.from_product(columns)
Output_Matrix =pd.DataFrame(index= Indices,columns=Columns)
Output_Matrix

DCT                     PCA                 \
                             Accuracy ProcessingTime Accuracy ProcessingTime   
K-means-Clustering 1              NaN            NaN      NaN            NaN   
                   4              NaN            NaN      NaN            NaN   
                   16             NaN            NaN      NaN            NaN   
GMM                1              NaN            NaN      NaN            NaN   
                   4              NaN            NaN      NaN            NaN   
                   16             NaN            NaN      NaN            NaN   
SVM                Linear         NaN            NaN      NaN            NaN   
                   NonLinear      NaN            NaN      NaN            NaN   

                             SelectKBest                 
                                Accuracy ProcessingTime  
K-means-Clustering 1                 NaN            NaN  
                   4                 NaN            NaN  
                   16                NaN            NaN  
GMM                1                 NaN            NaN  
                   4                 NaN            NaN  
                   16                NaN            NaN  
SVM                Linear            NaN            NaN  
                   NonLinear         NaN            NaN

In [ ]:
N_Clusters_Class=[1,4,16]

Features=[
    [X_train_DCT,X_test_DCT,'DCT'],
    [X_train_pca,X_test_pca,'PCA'],
    [X_train_SKB,X_test_SKB,'SelectKBest']
]

bestAcc=0
bestPredict=np.nan

for X,X_test,model in Features:
    for n_clusters in N_Clusters_Class:
        t1=time.time()
        GMM = GaussianMixture(n_components=n_clusters*10, random_state=0).fit(X)
        t2=time.time()-t1

        labels=GMM.predict(X)
        reference_labels={i: pd.value_counts(y_train[ labels==i].ravel()).index[0] for i in np.unique(labels)}
        TestLabels=GMM.predict(X_test)
        y_p=[reference_labels[i] for i in TestLabels]
        Acc=(y_p==y_test.ravel()).sum()/X_test.shape[0]
        if Acc>bestAcc:
            bestAcc=Acc
            bestPredict=y_p
            

        Output_Matrix.loc[('GMM',n_clusters),(model,'ProcessingTime')]=t2
        Output_Matrix.loc[('GMM',n_clusters),(model,'Accuracy')]=Acc

In [ ]:
Output_Matrix

DCT                     PCA                 \
                             Accuracy ProcessingTime Accuracy ProcessingTime   
K-means-Clustering 1              NaN            NaN      NaN            NaN   
                   4              NaN            NaN      NaN            NaN   
                   16             NaN            NaN      NaN            NaN   
GMM                1            0.382         56.447    0.666        36.6011   
                   4            0.535        81.9039   0.3275        18.9483   
                   16          0.3555        100.471    0.933        78.2821   
SVM                Linear         NaN            NaN      NaN            NaN   
                   NonLinear      NaN            NaN      NaN            NaN   

                             SelectKBest                 
                                Accuracy ProcessingTime  
K-means-Clustering 1                 NaN            NaN  
                   4                 NaN            NaN  
                   16                NaN            NaN  
GMM                1               0.452        23.7754  
                   4              0.4735        186.682  
                   16              0.768        80.8268  
SVM                Linear            NaN            NaN  
                   NonLinear         NaN            NaN

In [ ]:
print(bestAcc)
confusion_matrix(y_test,bestPredict)

0.933


array([[196,   0,   0,   1,   0,   0,   1,   0,   2,   0],
       [  0, 200,   0,   0,   0,   0,   0,   0,   0,   0],
       [  4,   1, 178,   1,   0,   0,   0,   2,  14,   0],
       [  0,   0,   0, 177,   0,   4,   0,   1,  18,   0],
       [  0,   1,   0,   0, 179,   0,   0,   1,   2,  17],
       [  1,   0,   0,  11,   0, 172,   3,   0,  13,   0],
       [  1,   0,   0,   0,   0,   0, 199,   0,   0,   0],
       [  0,   0,   5,   0,   0,   0,   0, 191,   0,   4],
       [  0,   0,   0,   0,   0,   5,   0,   1, 194,   0],
       [  0,   0,   0,   1,   0,   0,   0,  17,   2, 180]], dtype=int64)

In [ ]:
kernel = [['Linear','linear'],['NonLinear','rbf']]
for X,X_test,model in Features:
    for K_type,k in kernel:
        t1=time.time()
        clf = SVC(kernel = k,max_iter= 2000)
        clf.fit(X, y_train)  
        t2=time.time()-t1
        y_train = y_train.ravel()
        y_pred_train = clf.predict(X)
        y_pred_test = clf.predict(X_test)
        acc = accuracy_score(y_test, y_pred_test)
        

        Output_Matrix.loc[('SVM',K_type),(model,'ProcessingTime')]=t2
        Output_Matrix.loc[('SVM',K_type),(model,'Accuracy')]=acc
        print("For SVM with feature ", model , " :")
        print("Confusion matrix for training is ")
        cm = confusion_matrix(y_train, y_pred_train)
        print(cm)
        print("Train Accuracy  " , accuracy_score(y_train, y_pred_train))
        print("Confusion matrix for testing is ")
        cm = confusion_matrix(y_test, y_pred_test)
        print(cm)
        print("Test Accuracy  " , accuracy_score(y_test, y_pred_test))
        
        
Output_Matrix

C:\Users\lenovo\anaconda\lib\site-packages\sklearn\svm\_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


For SVM with feature  DCT  :
Confusion matrix for training is 
[[975   0   5   0   2  18   0   0   0   0]
 [  0 997   0   0   0   0   0   0   3   0]
 [ 15  11 854  58  12   1  13  10  21   5]
 [ 26  10  65 756   0  52   0  12  73   6]
 [  0   5   8   1 944   1   1   1   5  34]
 [ 21   3  12 126  10 802  17   0   7   2]
 [  5   1  56   0   2  10 926   0   0   0]
 [  2   0  30   9  10   0   0 904   1  44]
 [  8  20  40 159   4 103   2  10 633  21]
 [  8   8  13  20 105  16   1  68  48 713]]
Train Accuracy   0.8504
Confusion matrix for testing is 
[[193   0   0   1   0   3   2   1   0   0]
 [  0 194   1   0   0   0   0   1   4   0]
 [  6   1 151  22   1   3   6   3   7   0]
 [  5   0  12 149   0   6   0   6  21   1]
 [  3   0   4   2 185   1   1   2   0   2]
 [  3   2   2  13   7 165   1   1   5   1]
 [  2   0  13   0   3   4 178   0   0   0]
 [  1   0   8   1   2   1   0 178   0   9]
 [  5   4   5  28   4  42   2   1 103   6]
 [  0   0   0   1  30   4   0  13   8 144]]
Test Accuracy   0.

C:\Users\lenovo\anaconda\lib\site-packages\sklearn\svm\_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


For SVM with feature  PCA  :
Confusion matrix for training is 
[[1000    0    0    0    0    0    0    0    0    0]
 [   0 1000    0    0    0    0    0    0    0    0]
 [   0    0 1000    0    0    0    0    0    0    0]
 [   0    0    0 1000    0    0    0    0    0    0]
 [   0    0    0    0 1000    0    0    0    0    0]
 [   0    0    0    0    0 1000    0    0    0    0]
 [   0    0    0    0    0    0 1000    0    0    0]
 [   0    0    0    0    0    0    0 1000    0    0]
 [   0    0    0    0    0    0    0    0 1000    0]
 [   0    0    0    0    0    0    0    0    0 1000]]
Train Accuracy   1.0
Confusion matrix for testing is 
[[198   0   0   1   0   1   0   0   0   0]
 [  0 198   0   0   0   0   0   0   2   0]
 [  4   0 181   4   1   0   2   2   6   0]
 [  1   0   3 179   0   7   0   0   6   4]
 [  2   0   2   0 190   0   0   2   0   4]
 [  2   3   1   2   8 175   2   0   7   0]
 [  3   0   1   0   0   3 193   0   0   0]
 [  0   0   5   0   2   0   0 188   0   5]
 [  1   

C:\Users\lenovo\anaconda\lib\site-packages\sklearn\svm\_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


For SVM with feature  SelectKBest  :
Confusion matrix for training is 
[[1000    0    0    0    0    0    0    0    0    0]
 [   0 1000    0    0    0    0    0    0    0    0]
 [   0    0 1000    0    0    0    0    0    0    0]
 [   3    2    6  899    0   64    0    0   26    0]
 [   2    0    0    0  898    0    0    0    0  100]
 [  10    7    4   54    4  910    2    0    9    0]
 [   0    0    0    0    0    0 1000    0    0    0]
 [   0    2    0   15    5    1    0  746    0  231]
 [   5    2    3   24    5   15    2    2  941    1]
 [   6    4    8    8  201    1    1   62    2  707]]
Train Accuracy   0.9101
Confusion matrix for testing is 
[[192   0   0   1   2   2   2   0   1   0]
 [  0 196   0   0   0   0   0   0   4   0]
 [  4   0 174   5   5   0   3   2   7   0]
 [  1   2   3 164   0  15   0   0  10   5]
 [  2   0   1   0 170   0   3   3   0  21]
 [  3   4   3   5   8 162   4   0   9   2]
 [  8   0   2   0   1   3 186   0   0   0]
 [  0   0   8   5   3   0   0 142   0  4

DCT                     PCA                 \
                             Accuracy ProcessingTime Accuracy ProcessingTime   
K-means-Clustering 1              NaN            NaN      NaN            NaN   
                   4              NaN            NaN      NaN            NaN   
                   16             NaN            NaN      NaN            NaN   
GMM                1            0.382         56.447    0.666        36.6011   
                   4            0.535        81.9039   0.3275        18.9483   
                   16          0.3555        100.471    0.933        78.2821   
SVM                Linear        0.82        7.84604   0.9325        6.73797   
                   NonLinear    0.951        7.89289   0.9765        13.4839   

                             SelectKBest                 
                                Accuracy ProcessingTime  
K-means-Clustering 1                 NaN            NaN  
                   4                 NaN            NaN  
                   16                NaN            NaN  
GMM                1               0.452        23.7754  
                   4              0.4735        186.682  
                   16              0.768        80.8268  
SVM                Linear          0.849        7.67646  
                   NonLinear       0.953        10.9268

In [ ]:

from sklearn.cluster import KMeans
for X,X_test,model in Features:
    print("For K-means-Clustering with feature ", model , " :")
    for n_clusters in N_Clusters_Class:
        t1=time.time()
        kmeans = KMeans(n_clusters=n_clusters*10)
        kmeans.fit(X, y_train)  
        t2=time.time()-t1
        y = np.array(y_train)
        labels=kmeans.labels_
        Ladels_correct = {}
        for i in np.unique(labels):
            values_cluster = pd.value_counts(y[labels==i].ravel())
            Ladels_correct[i] = values_cluster.index[0]

            
        y_pred_train = kmeans.predict(X)
        y_predict_train =[Ladels_correct[i] for i in y_pred_train]
        print("number of clusters for each class : ", n_clusters )
        print("Confusion matrix for training is ")
        cm = confusion_matrix(y_train, y_predict_train)
        print(cm)
        print("Train Accuracy  " , accuracy_score(y_train, y_predict_train))
        
        y_pred = kmeans.predict(X_test)
        y_predict=[Ladels_correct[i] for i in y_pred]
        print("Confusion matrix for testing is ")
        cm = confusion_matrix(y_test, y_predict)
        print(cm)
        acc =  accuracy_score(y_test, y_predict)
        print("Test Accuracy  " ,acc)
        
        Output_Matrix.loc[('K-means-Clustering',n_clusters),(model,'ProcessingTime')]=t2
        Output_Matrix.loc[('K-means-Clustering',n_clusters),(model,'Accuracy')]= acc
        
        
Output_Matrix

For K-means-Clustering with feature  DCT  :
number of clusters for each class :  1
Confusion matrix for training is 
[[796  14   2  11   4  96   1  10  49  17]
 [  0 965   1   9   0  21   2   1   1   0]
 [ 19  66 625 129  16  12  32  23  58  20]
 [  0  99  31 609   1  10   4   5 201  40]
 [  4  48   2   7 538 153  30   1   8 209]
 [ 19  56   9 180   2 389  26  23 242  54]
 [  1  72   3   1 169  32 717   0   5   0]
 [ 16  63   4  21  11  31   1 777  13  63]
 [ 18  53   5 127  15  68   8   4 607  95]
 [ 24  38   0  17  54  78   1  26  16 746]]
Train Accuracy   0.6769
Confusion matrix for testing is 
[[168   1   0   0   0  28   0   0   2   1]
 [  0 184   0   2   0  14   0   0   0   0]
 [ 12   1 123  16   2   7   8   2  23   6]
 [  1  11   4 111   0   5   0   2  50  16]
 [  1   2   0   0 124  37   3   0   0  33]
 [  4   9   0  16   2 114   2   2  47   4]
 [  0   9   0   0  23   6 162   0   0   0]
 [  3   0   3   5   0   8   0 165   4  12]
 [ 10   3   0  10   3  21   2   0 125  26]
 [  3   

number of clusters for each class :  16
Confusion matrix for training is 
[[963   1   3   1   3   0  19   0   9   1]
 [  0 979   5   3   1   1   4   1   3   3]
 [  9  11 882   4  14   1  20  10  41   8]
 [  6   6  12 810   6  95   6  12  28  19]
 [  4   9   0   0 682   1  16   1   3 284]
 [  6   5   3 120  34 731  31   3  17  50]
 [ 13   3   1   0   2   1 975   0   2   3]
 [  1  21   6   2  22   0   0 898   4  46]
 [ 16  39  13  13  11  10  21   2 856  19]
 [  5   6   0   9 200  13   3  22   9 733]]
Train Accuracy   0.8509
Confusion matrix for testing is 
[[196   0   0   0   0   1   2   0   1   0]
 [  0 199   0   0   0   0   0   0   1   0]
 [  5   2 169   3   3   1   3   4  10   0]
 [  1   2   1 157   0  27   0   1   9   2]
 [  0   0   0   0 141   2   5   0   2  50]
 [  1   1   2  13   3 165   8   0   3   4]
 [  4   0   0   1   1   0 193   0   0   1]
 [  0   0   5   0   3   0   0 190   0   2]
 [  2   4   0   0   5   2   5   0 179   3]
 [  0   0   0   0  36   0   0   5   1 158]]
Test Ac

DCT                     PCA                 \
                             Accuracy ProcessingTime Accuracy ProcessingTime   
K-means-Clustering 1           0.7195        3.48667   0.6675        4.53886   
                   4            0.846        8.37061    0.886        9.42379   
                   16          0.8935        22.8858   0.9305        27.4784   
GMM                1            0.382         56.447    0.666        36.6011   
                   4            0.535        81.9039   0.3275        18.9483   
                   16          0.3555        100.471    0.933        78.2821   
SVM                Linear        0.82        7.84604   0.9325        6.73797   
                   NonLinear    0.951        7.89289   0.9765        13.4839   

                             SelectKBest                 
                                Accuracy ProcessingTime  
K-means-Clustering 1               0.749        3.05583  
                   4                0.83        9.90051  
                   16             0.8735        25.8309  
GMM                1               0.452        23.7754  
                   4              0.4735        186.682  
                   16              0.768        80.8268  
SVM                Linear          0.849        7.67646  
                   NonLinear       0.953        10.9268

In [ ]:
Output_Matrix

DCT                     PCA                 \
                             Accuracy ProcessingTime Accuracy ProcessingTime   
K-means-Clustering 1           0.7195        3.48667   0.6675        4.53886   
                   4            0.846        8.37061    0.886        9.42379   
                   16          0.8935        22.8858   0.9305        27.4784   
GMM                1            0.382         56.447    0.666        36.6011   
                   4            0.535        81.9039   0.3275        18.9483   
                   16          0.3555        100.471    0.933        78.2821   
SVM                Linear        0.82        7.84604   0.9325        6.73797   
                   NonLinear    0.951        7.89289   0.9765        13.4839   

                             SelectKBest                 
                                Accuracy ProcessingTime  
K-means-Clustering 1               0.749        3.05583  
                   4                0.83        9.90051  
                   16             0.8735        25.8309  
GMM                1               0.452        23.7754  
                   4              0.4735        186.682  
                   16              0.768        80.8268  
SVM                Linear          0.849        7.67646  
                   NonLinear       0.953        10.9268